In [13]:
pip install flask


In [3]:
pip install langchain


In [4]:
pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Successfully uninstalled langchain-core-0.3.31
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.15
    Uninstalling langchain-0.3.15:
      Successfully uninstalled langchain-0.3.15


In [5]:
pip install openai


In [6]:
pip install faiss-cpu  # or faiss-gpu if you have GPU support


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 34.9 MB/s eta 0:00:00


In [7]:
pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [8]:
pip install beautifulsoup4 requests


In [9]:
pip install python-dotenv


In [10]:
pip install flask langchain openai faiss-cpu sentence-transformers beautifulsoup4 requests python-dotenv


In [20]:
pip install faiss-cpu


In [37]:
pip install gunicorn pyngrok


In [45]:
pip install flask selenium webdriver-manager langchain faiss-cpu transformers pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 35.9 MB/s eta 0:00:00


In [48]:
!sudo apt install -y chromium-browser

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor libfuse3-3 liblzo2-2 snapd squashfs-tools systemd-hwe-hwdb udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
0 upgraded, 8 newly installed, 0 to remove and 18 not upgraded.
Need to get 30.1 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.4 [598 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3build1 [159 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 udev amd64 249.11-0ubu

In [ ]:
# api key - sk-proj-7zM7fxSz893IEeP481o7Kk2yEcY0MJnKqC9cg7wqT51wM930pdzDVILvChsgvl-CEk4CKLxapRT3BlbkFJDnt2jTZblgH5vBSAFMxq2lK6BwE0G4_K5uEupwrTxeHhSmjn_dKi8h0_nYsAA
# URL = "https://brainlox.com/courses/category/technical"
# ngrok_auth_token = "2sLHxLr6pR4N5vBZwV2XB33udBd_4683LAcSLYnqkpJ1k8esx"

In [61]:
import os
import json
from flask import Flask, request, jsonify
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document

# ✅ Securely load OpenAI API key
openai_api_key = "sk-proj-7zM7fxSz893IEeP481o7Kk2yEcY0MJnKqC9cg7wqT51wM930pdzDVILvChsgvl-CEk4CKLxapRT3BlbkFJDnt2jTZblgH5vBSAFMxq2lK6BwE0G4_K5uEupwrTxeHhSmjn_dKi8h0_nYsAA"

if not openai_api_key:
    raise ValueError("❌ OpenAI API key is missing.")
else:
    print("✅ OpenAI API Key loaded successfully.")

# ✅ Step 1: Extract and Validate Data from Brainlox
URL = "https://brainlox.com/courses/category/technical"

try:
    loader = WebBaseLoader(URL)
    raw_documents = loader.load()

    if not raw_documents:
        raise ValueError("❌ No documents loaded. The website might require JavaScript rendering.")

    # Ensure we load at least 10 documents
    documents = [
        Document(page_content=doc.page_content, metadata={"source": doc.metadata.get("source", URL)})
        for doc in raw_documents[:10]  # Only take first 10 docs
    ]

    print(f"✅ Loaded {len(documents)} documents from {URL}")

except Exception as e:
    print(f"❌ Error loading documents: {e}")
    documents = []  # Ensure documents is always defined

if not documents:
    raise ValueError("❌ No valid documents found. Try another scraping method.")

# ✅ Step 2: Initialize FAISS Vector Store
try:
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    texts = [doc.page_content for doc in documents]

    # Store in FAISS
    db = FAISS.from_texts(texts, embeddings)
    db.save_local("faiss_index")  # ✅ Save FAISS index
    print("✅ FAISS index created and saved.")

    db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    retriever = db.as_retriever()
    print("✅ FAISS vector store loaded successfully.")

except Exception as e:
    print(f"❌ Error initializing FAISS: {e}")
    raise

# ✅ Step 3: Initialize LangChain Chat Model
try:
    llm = OpenAI(api_key=openai_api_key, temperature=0.5)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory
    )

    print("✅ LangChain Chat Model initialized successfully.")

except Exception as e:
    print(f"❌ Error initializing ConversationalRetrievalChain: {e}")
    chain = None

# ✅ Step 4: Deploy Flask Chatbot with Ngrok
from pyngrok import ngrok

ngrok.set_auth_token("2sLHxLr6pR4N5vBZwV2XB33udBd_4683LAcSLYnqkpJ1k8esx")

public_url = ngrok.connect(5000)
print(f"🌍 Public URL: {public_url}")

# ✅ Step 5: Initialize Flask App
app = Flask(__name__)

@app.route("/chat", methods=["POST"])
def chatbot_response():
    """Handles chatbot conversation."""
    if not chain:
        return jsonify({"error": "Chatbot is disabled. Set up an OpenAI API key or use a local LLM."}), 400

    data = request.json
    user_input = data.get("message")

    if not user_input:
        return jsonify({"error": "Message is required"}), 400

    try:
        response = chain.run({"question": user_input})
        return jsonify({
            "response": response,
            "chat_history": memory.chat_memory.messages  # ✅ Returns conversation history
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    print(f"🚀 Chatbot server is running on: {public_url}")
    app.run(host="0.0.0.0", port=5000, debug=True)  # ✅ Debug Mode ON


✅ OpenAI API Key loaded successfully.
✅ Loaded 1 documents from https://brainlox.com/courses/category/technical
✅ FAISS index created and saved.
✅ FAISS vector store loaded successfully.
✅ LangChain Chat Model initialized successfully.
🌍 Public URL: NgrokTunnel: "https://4be6-34-48-26-94.ngrok-free.app" -> "http://localhost:5000"
🚀 Chatbot server is running on: NgrokTunnel: "https://4be6-34-48-26-94.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
